
# Step-by-step agent negotiation


In this section, you will see how the negotiation works in our framework.

## The big picture


The following diagram gives you a high-level understanding on how a generic negotiation works.

.. mermaid:: ../diagrams/negotiation-guide/summary.mmd

Let's see step by step what happens:

1. `Agent_1` calls `get_service_description(is_supply)` to generate the service description. `is_supply` is a flag to
    switch between the seller and buyer roles.
2. `Agent_1` sends a `register_service` request to the OEF node, so she can be advertised on the OEF.
3. Analogous to (1), but for `Agent_2`
4. Analogous to (2), but for `Agent_2`
5. `Agent_1` calls `build_services_query(is_searching_for_sellers)` to generate a `query` for the OEF
6. `Agent_1` send a `search_service` request with the `query` previously generated.
7. The OEF node returns a search result with the list of agent ids matching the `query`
8. `Agent_1` finds `Agent_2`, so `Agent_1` sends a `CFP` to `Agent_2`, meaning that she wants to start a negotiation.
9. `Agent_2` calls `get_proposals()` to generate a proposal for answering `Agent_1`
10. `Agent_2` replies with a `Propose` message as an answer for the `CFP`.
11. `Agent_1` sends an `Accept` message to `Agent_2`, meaning that she accepts the proposal.
12. `Agent_1` sends a Transaction request to the `Controller`.
13. `Agent_2` replies with a _matched accept_ to `Agent_1`, meaning that she confirms definitively the transaction. 
14. `Agent_2` sends a Transaction request to the `Controller` (analogous to step 12).


## Analyzing the APIs

In the following, we're going to describe the steps listed before, but more in detail, using code examples from the framework.

### Instantiate an agent


In [3]:
from tac.agents.v1.examples.baseline import BaselineAgent
from tac.agents.v1.examples.strategy import BaselineStrategy

strategy = BaselineStrategy()
agent = BaselineAgent(name="tac_agent", oef_addr="127.0.0.1", oef_port=10000, strategy=strategy)





### Registration

This part covers the steps 1-4. That is, when the agents build their own description and register as a service to the OEF.
This step allows the agents to be found via search queries, and hence increasing the probability to be found by other agents. 


#### The `get_service_description(is_supply)` method

This method generates a `Description` object of the [Python OEF SDK](https://github.com/fetchai/oef-sdk-python.git) 
(check the documentation [here](http://oef-sdk-docs.fetch.ai/oef.html#oef.schema.Description)).
It is basically a data structure that holds a dictionary objects, mapping from attribute names (strings) to some values.
Moreover, it might refer to a [DataModel](http://oef-sdk-docs.fetch.ai/oef.html#oef.schema.DataModel) object, which defines
the abstract structure that a `Description` object should have. You can think of them in terms of the relational database
domain: a `DataModel` object corresponds to an SQL Table, whereas a `Description` object correspond to a row of that table.  

In the context of TAC, the `Description` for service registration looks like the following:


In [7]:
from oef.schema import Description


description = Description({
    'tac_good_0_pbk': 1, 
    'tac_good_1_pbk': 1, 
    'tac_good_2_pbk': 1, 
    'tac_good_3_pbk': 1, 
    'tac_good_4_pbk': 1, 
    'tac_good_5_pbk': 1, 
    'tac_good_6_pbk': 1, 
    'tac_good_7_pbk': 1, 
    'tac_good_8_pbk': 1, 
    'tac_good_9_pbk': 1
}, data_model=None)



The attribute names `tac_good_X_pbk` is the name given to each tradable good.  

The argument `data_model` is set to `None`, but in the framework is properly set.

Notice that the keys are automatically generated, depending on the number of goods in the game.  
